In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient


import numpy as np
import pandas as pd
import scipy.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [2]:
client = MongoClient()
db_nba= client.nba
collection = db_nba.player_stats
df_nba = pd.DataFrame(list(collection.find()))

Let's take a high-level look at our dataframe.

In [3]:
df_nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17675 entries, 0 to 17674
Data columns (total 66 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   17675 non-null  object 
 1   GROUP_SET             17675 non-null  object 
 2   GROUP_VALUE           17675 non-null  object 
 3   TEAM_ID               17675 non-null  int64  
 4   TEAM_ABBREVIATION     17675 non-null  object 
 5   MAX_GAME_DATE         17675 non-null  object 
 6   GP                    17675 non-null  int64  
 7   W                     17675 non-null  int64  
 8   L                     17675 non-null  int64  
 9   W_PCT                 17675 non-null  float64
 10  MIN                   17675 non-null  float64
 11  FGM                   17675 non-null  int64  
 12  FGA                   17675 non-null  int64  
 13  FG_PCT                17675 non-null  float64
 14  FG3M                  17675 non-null  int64  
 15  FG3A               

We won't need any of the ranking values, so let's get rid of those. 

In [7]:
remove = []
for col in df_nba.columns:
    if 'RANK' in col:
        remove.append(col)
        
df_nba.drop(columns = remove, inplace=True)

In [8]:
df_nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17675 entries, 0 to 17674
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                17675 non-null  object 
 1   GROUP_SET          17675 non-null  object 
 2   GROUP_VALUE        17675 non-null  object 
 3   TEAM_ID            17675 non-null  int64  
 4   TEAM_ABBREVIATION  17675 non-null  object 
 5   MAX_GAME_DATE      17675 non-null  object 
 6   GP                 17675 non-null  int64  
 7   W                  17675 non-null  int64  
 8   L                  17675 non-null  int64  
 9   W_PCT              17675 non-null  float64
 10  MIN                17675 non-null  float64
 11  FGM                17675 non-null  int64  
 12  FGA                17675 non-null  int64  
 13  FG_PCT             17675 non-null  float64
 14  FG3M               17675 non-null  int64  
 15  FG3A               17675 non-null  int64  
 16  FG3_PCT            176

In [9]:
df_nba.head()

,_id,GROUP_SET,GROUP_VALUE,TEAM_ID,TEAM_ABBREVIATION,MAX_GAME_DATE,GP,W,L,W_PCT,...,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,CFID,CFPARAMS
0,5eb88bb35e7937b47b59984a,By Year,2002-03,1610612738,BOS,2003-04-16T00:00:00,41,21,20,0.512,...,6,62,0,72,-58,220.1,0,0,264,2002-03
1,5eb88bb35e7937b47b59984b,By Year,2001-02,1610612763,MEM,2002-04-05T00:00:00,66,18,48,0.273,...,16,123,0,417,-384,1020.2,0,0,264,2001-02
2,5eb88bb35e7937b47b59984c,By Year,2000-01,1610612763,VAN,2001-04-18T00:00:00,66,19,47,0.288,...,12,160,1,396,-220,1048.3,1,0,264,2000-01
3,5eb88bb35e7937b47b59984d,By Year,1999-00,1610612763,VAN,2000-03-21T00:00:00,42,13,29,0.310,...,7,108,31,203,-62,664.3,0,0,264,1999-00
4,5eb88bb35e7937b47b59984e,By Year,1998-99,1610612737,ATL,1999-05-05T00:00:00,50,31,19,0.620,...,25,143,2,489,205,1068.7,5,0,264,1998-99
